# Integrating Bike Share Data Sets

In [154]:
# importing libraries
import numpy as np
import pandas as pd
import datetime
import re
from IPython.display import display

# function

In [155]:
def compare_value(value1, value2):
    df_val1=pd.DataFrame(value1, columns=['value1'])
    df_val2=pd.DataFrame(value2, columns=['value2'])
    tmp = df_val1.merge(df_val2, right_index=True, left_index=True, how='outer')
    tmp['compare'] = tmp['value1'] == tmp['value2']
    return tmp

## I. Examin two data sets

In [156]:
# read in two data set
df1 = pd.read_csv('bikeshare_data_S1.csv')
df2 = pd.read_csv('bikeshare_data_S2.csv')
# Display two dataframes together to compare
print 'S1:' + str(df1.shape)
display(df1.head(3))
print 'S2:' + str(df2.shape)
display(df2.head(3))

S1:(8500, 10)


,Duration (ms),Start date,Start station number,Start station,End station number,End station,Bike number,Member type,Helmet rented,Credit card
0,840866,8/31/2016 23:59,31117,15th & Euclid St NW,31228,8th & H St NW,W20409,Registered,1,**** **** **** 7976
1,656098,8/31/2016 23:58,31279,19th & G St NW,31600,5th & K St NW,W20756,Casual,0,NaN
2,353159,8/31/2016 23:58,31107,Lamont & Mt Pleasant NW,31101,14th & V St NW,W22626,Registered,0,NaN


S2:(2000, 8)


,time_start,time_end,origin_station,destination_station,bike,member,helmet,payment_method
0,10:29:00 2016-08-31,10:47:56 2016-08-31,31102,31279,20978,No,-,credit card
1,10:29:00 2016-08-31,10:32:12 2016-08-31,31505,31627,20515,Yes,-,cash
2,10:29:00 2016-08-31,10:32:16 2016-08-31,31400,31401,20108,Yes,-,cash


In [122]:
print df1.info()
print df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 10 columns):
Duration (ms)           8500 non-null int64
Start date              8500 non-null object
Start station number    8500 non-null int64
Start station           8500 non-null object
End station number      8500 non-null int64
End station             8500 non-null object
Bike number             8500 non-null object
Member type             8500 non-null object
Helmet rented           8500 non-null int64
Credit card             4288 non-null object
dtypes: int64(4), object(6)
memory usage: 664.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
time_start             2000 non-null object
time_end               2000 non-null object
origin_station         2000 non-null int64
destination_station    2000 non-null int64
bike                   2000 non-null int64
member                 2000 non-null object
helmet                 

## II. Resolve schema conflict

In [123]:
compare_value(df1.dtypes, df2.dtypes)

,value1,value2,compare
0,int64,int64,True
1,int64,int64,True
2,int64,int64,True
3,int64,object,False
4,object,object,True
5,object,object,True
6,object,object,True
7,object,object,True
8,object,NaN,False
9,object,NaN,False


### Schema conflict 1: S2 does not have duration, S1 does not have time_end, datetime format differs in S1 and S2
- calculate duration for S2
- del time_end from S2
- convert S1 duration to seconds
- make consistent the datetime format

In [124]:
df2['time_end'] = pd.to_datetime(df2['time_end'])
df2['time_start'] = pd.to_datetime(df2['time_start'])

# define a funciton to calculate duration (seconds) from two datetime object
def calculateDuration(startTime, endTime):
    return (endTime - startTime).total_seconds()

df2['duration'] = df2.apply(lambda row: calculateDuration(row['time_start'],row['time_end']),axis = 1)

del df2['time_end']

df2['duration'].head()

0    1136.0
1     192.0
2     196.0
3    1403.0
4     648.0
Name: duration, dtype: float64

In [125]:
# covert Duration (ms) in S1 to seconds, and rename to be consistent
df1['Duration (ms)'] = df1['Duration (ms)'].apply(lambda x: round(x/1000))

df1.rename(columns = {'Duration (ms)':'duration'}, inplace = True)

df1['duration'].head()

0    840.0
1    656.0
2    353.0
3    219.0
4    213.0
Name: duration, dtype: float64

In [126]:
df1['Start date'] = pd.to_datetime(df1['Start date'])
df1.rename(columns = {'Start date':'time_start'}, inplace = True)
df1.time_start.head()

0   2016-08-31 23:59:00
1   2016-08-31 23:58:00
2   2016-08-31 23:58:00
3   2016-08-31 23:58:00
4   2016-08-31 23:56:00
Name: time_start, dtype: datetime64[ns]

In [127]:
df2.time_start.head()

0   2016-08-31 10:29:00
1   2016-08-31 10:29:00
2   2016-08-31 10:29:00
3   2016-08-31 10:29:00
4   2016-08-31 10:28:00
Name: time_start, dtype: datetime64[ns]

In [128]:
df1.head(2)

,duration,time_start,Start station number,Start station,End station number,End station,Bike number,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,15th & Euclid St NW,31228,8th & H St NW,W20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,19th & G St NW,31600,5th & K St NW,W20756,Casual,0,NaN


In [129]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 10 columns):
duration                8500 non-null float64
time_start              8500 non-null datetime64[ns]
Start station number    8500 non-null int64
Start station           8500 non-null object
End station number      8500 non-null int64
End station             8500 non-null object
Bike number             8500 non-null object
Member type             8500 non-null object
Helmet rented           8500 non-null int64
Credit card             4288 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 664.1+ KB


In [130]:
df2.head(2)

,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,No,-,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Yes,-,cash,192.0


### Schema conflict 2: S2 missing station names
- check whether station number and name has one to one corrwspondence, if yes
- del Start station and End station from S1

In [131]:
print 'S1:' + str(df1.shape)
display(df1.head(2))
print 'S2:' + str(df2.shape)
display(df2.head(2))

S1:(8500, 10)


,duration,time_start,Start station number,Start station,End station number,End station,Bike number,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,15th & Euclid St NW,31228,8th & H St NW,W20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,19th & G St NW,31600,5th & K St NW,W20756,Casual,0,NaN


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,No,-,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Yes,-,cash,192.0


In [132]:
# check whether there is a one-to-one correspondance in these two columns
station = df1[['Start station number','Start station']].drop_duplicates(keep='last')
station['Start station number'].value_counts()

31231    1
31411    1
31401    1
31914    1
31509    1
31404    1
31615    1
31406    1
31402    1
31408    1
31518    1
31282    1
31910    1
31624    1
31124    1
31512    1
31614    1
31400    1
31089    1
31029    1
31236    1
31609    1
31912    1
31909    1
32024    1
31244    1
31121    1
31635    1
32014    1
31022    1
        ..
31601    1
31090    1
31091    1
31092    1
31093    1
31094    1
31073    1
31072    1
31071    1
31069    1
31050    1
31051    1
31052    1
31053    1
31054    1
31055    1
31056    1
31057    1
31058    1
31234    1
31060    1
31501    1
31062    1
31063    1
31064    1
31065    1
31066    1
31067    1
31068    1
31232    1
Name: Start station number, Length: 377, dtype: int64

In [133]:
# As there is always one-to-one correpsondency, we choose to drop name columns in S1 to avoid repeated information
del df1['Start station']
del df1['End station']
df1.rename(columns = {'Start station number':'origin_station', 'End station number':'destination_station'}, inplace = True)
df1.head(2)

,duration,time_start,origin_station,destination_station,Bike number,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,W20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,W20756,Casual,0,NaN


### Schema conflict 3: Bike number format differs in S1 and S2
- check whether the pattern is consistent in S1, if yes, delete letter 'W' in bike number in S1

In [134]:
print 'S1:' + str(df1.shape)
display(df1.head(2))
print 'S2:' + str(df2.shape)
display(df2.head(2))

S1:(8500, 8)


,duration,time_start,origin_station,destination_station,Bike number,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,W20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,W20756,Casual,0,NaN


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,No,-,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Yes,-,cash,192.0


In [135]:
# define a functin to check the pattern of bike number column
def check_bikeNumPattern(value, pattern=re.compile('W\d{5}')):
    return 'OK' if pattern.match(value) else 'Failed'

# Checking whether all rows in the Bike number column 
df1['test_bikeNum'] = df1['Bike number'].apply(check_bikeNumPattern)
print len(df1[df1['test_bikeNum']!='OK'])
df1[df1['test_bikeNum']!='OK']# return nothing only when all rows are consistent with the pattern observed

9


,duration,time_start,origin_station,destination_station,Bike number,Member type,Helmet rented,Credit card,test_bikeNum
1016,232.0,2016-08-31 20:29:00,31077,31031,w00644,Registered,1,**** **** **** 1916,Failed
4588,436.0,2016-08-31 17:17:00,31037,31035,w00223,Registered,0,NaN,Failed
5143,532.0,2016-08-31 16:52:00,31236,31014,w00583,Registered,2,NaN,Failed
5747,395.0,2016-08-31 16:01:00,31037,31901,w01117,Casual,0,**** **** **** 6479,Failed
6033,4235.0,2016-08-31 15:30:00,31042,31077,w00644,Casual,0,NaN,Failed
6108,1041.0,2016-08-31 15:17:00,31205,31236,w00583,Registered,0,**** **** **** 2402,Failed
6307,512.0,2016-08-31 14:48:00,31047,31042,w00644,Registered,0,**** **** **** 5775,Failed
7118,346.0,2016-08-31 12:50:00,31234,31205,w00583,Registered,0,**** **** **** 5720,Failed
7217,735.0,2016-08-31 12:36:00,31104,31234,w00583,Casual,0,**** **** **** 4241,Failed


As there are only 9 rows across 8500 records, so I believe was just an inconsistency issue (some with capital 'W' and 9 with lower case 'w'), I decided to only maintain the digit as the bike number.

In [136]:
df1['Bike number'] = df1['Bike number'].str.findall('\d{5}').str.get(0)
df1.rename(columns = {'Bike number':'bike'}, inplace = True)
del df1['test_bikeNum']
df1.head(2)

,duration,time_start,origin_station,destination_station,bike,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,20756,Casual,0,NaN


### Schema conflict 4: Member type values differs in S1 and S2
- check unique vlaues of Member Type/member, convert values in S2 to Registered/Casual.

In [137]:
compare_value(df1['Member type'].unique(), df2['member'].unique())

,value1,value2,compare
0,Casual,No,False
1,Registered,Yes,False


In [138]:
print 'S1:' + str(df1.shape)
display(df1.head(5))
print 'S2:' + str(df2.shape)
display(df2.head(5))

S1:(8500, 8)


,duration,time_start,origin_station,destination_station,bike,Member type,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,20756,Casual,0,NaN
2,353.0,2016-08-31 23:58:00,31107,31101,22626,Registered,0,NaN
3,219.0,2016-08-31 23:58:00,31200,31212,00980,Casual,0,**** **** **** 9137
4,213.0,2016-08-31 23:56:00,31281,31280,21338,Registered,0,NaN


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,No,-,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Yes,-,cash,192.0
2,2016-08-31 10:29:00,31400,31401,20108,Yes,-,cash,196.0
3,2016-08-31 10:29:00,31261,31077,20307,No,1,credit card,1403.0
4,2016-08-31 10:28:00,31279,31201,1025,Yes,-,credit card,648.0


In [139]:
df1['Member type'].value_counts()

Registered    7118
Casual        1382
Name: Member type, dtype: int64

In [140]:
df2['member'].value_counts()

Yes    1832
No      168
Name: member, dtype: int64

In [141]:
df2.member.replace(['Yes','No'], ['Registered','Casual'], inplace=True)

# rename S1 member type column
df1.rename(columns = {'Member type':'member'}, inplace = True)

print 'S1:' + str(df1.shape)
display(df1.head(2))
print 'S2:' + str(df2.shape)
display(df2.head(2))

S1:(8500, 8)


,duration,time_start,origin_station,destination_station,bike,member,Helmet rented,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,20756,Casual,0,NaN


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,Casual,-,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Registered,-,cash,192.0


In [142]:
compare_value(df1['member'].unique(), df2['member'].unique())

,value1,value2,compare
0,Casual,Casual,True
1,Registered,Registered,True


### Schema conflict 5: Values in helmet differs in S1 and S2
- fill in missing values '-' in S2 with 0
- cast S2 helmet datatype to int

In [143]:
df1['Helmet rented'].value_counts()

0    5498
1    1714
2    1288
Name: Helmet rented, dtype: int64

In [144]:
df2.helmet.value_counts()

-    1289
1     412
2     299
Name: helmet, dtype: int64

In [145]:
df2.helmet.replace(['-'], [0], inplace=True)
df2.helmet = df2.helmet.astype(int)
df2.head()

df1.rename(columns = {'Helmet rented':'helmet'}, inplace = True)

print 'S1:' + str(df1.shape)
display(df1.head(2))
print 'S2:' + str(df2.shape)
display(df2.head(2))

S1:(8500, 8)


,duration,time_start,origin_station,destination_station,bike,member,helmet,Credit card
0,840.0,2016-08-31 23:59:00,31117,31228,20409,Registered,1,**** **** **** 7976
1,656.0,2016-08-31 23:58:00,31279,31600,20756,Casual,0,NaN


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,Casual,0,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Registered,0,cash,192.0


### Schema conflict 6: S1 stores credit card number, S2 stores payment_method
- Assumen Nan vlaues means user pay by cash, we fill in nan values in S1 with cash
- As there is no credit card number in S2, and consider there is not much information you could get from the last 4 digits of credit card number in analysis, we replace credit card number in S1 with string 'credit card'

In [146]:
df1['Credit card'] = df1['Credit card'].apply(lambda x: 'cash' if pd.isnull(x) else 'credit card')
df1.rename(columns = {'Credit card':'payment_method'}, inplace = True)

print 'S1:' + str(df1.shape)
display(df1.head(2))
print 'S2:' + str(df2.shape)
display(df2.head(2))

S1:(8500, 8)


,duration,time_start,origin_station,destination_station,bike,member,helmet,payment_method
0,840.0,2016-08-31 23:59:00,31117,31228,20409,Registered,1,credit card
1,656.0,2016-08-31 23:58:00,31279,31600,20756,Casual,0,cash


S2:(2000, 8)


,time_start,origin_station,destination_station,bike,member,helmet,payment_method,duration
0,2016-08-31 10:29:00,31102,31279,20978,Casual,0,credit card,1136.0
1,2016-08-31 10:29:00,31505,31627,20515,Registered,0,cash,192.0


## Adapt datatypes across schemas S1 and S2
We've complete data auditing for all the columns in two schemas S1 and S2. 
Now,we make all the datatypes consistent between S1 and S2 for data merging.

In [147]:
print df1.info()
print df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 8 columns):
duration               8500 non-null float64
time_start             8500 non-null datetime64[ns]
origin_station         8500 non-null int64
destination_station    8500 non-null int64
bike                   8500 non-null object
member                 8500 non-null object
helmet                 8500 non-null int64
payment_method         8500 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 531.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
time_start             2000 non-null datetime64[ns]
origin_station         2000 non-null int64
destination_station    2000 non-null int64
bike                   2000 non-null int64
member                 2000 non-null object
helmet                 2000 non-null int64
payment_method         2000 non-null object
duration               2000 no

In [148]:
print('df1 columns unique? {0}\ndf2 columns unique? {1}'.format(df1.columns.is_unique, df2.columns.is_unique)) 

df1 columns unique? True
df2 columns unique? True


In [149]:
df1_attr=set(df1.columns)
df2_attr=set(df2.columns)

In [150]:
df1_attr.difference(df2_attr)

set()

In [151]:
df1.columns.to_series()

duration                          duration
time_start                      time_start
origin_station              origin_station
destination_station    destination_station
bike                                  bike
member                              member
helmet                              helmet
payment_method              payment_method
dtype: object

In [152]:
# now adapt the datatypes in S1 to match S2
for y in df2.columns:
    if df1[y].dtype != df2[y].dtype:
        print "Column " + y + "in df1: "+ str(df1[y].dtype) + "to" + str(df2[y].dtype)
        df1[y] = df1[y].astype(df2[y].dtype) 

Column bikein df1: objecttoint64


Here, we've assumed a very simplistic case. However, you will need to be more careful to audit the datatype inconsistency when doing data integration. Also, during type casting, you might find more inconsistencies to be fixed before merging (very often) >.< So be careful! :)

In [153]:
compare_value(df1.dtypes, df2.dtypes)

,value1,value2,compare
0,float64,datetime64[ns],False
1,datetime64[ns],int64,False
2,int64,int64,True
3,int64,int64,True
4,int64,int64,True
5,int64,float64,False
6,object,object,True
7,object,object,True


## III. Merge Data

In [ ]:
df = pd.concat([df1,df2])
print df.shape
df.head()

## IV. Resolve data conflicts

In [ ]:
# check duplication
cols = df.columns.difference([]) # use all columns to check duplicates
duplicates = df[df.duplicated(cols,keep=False)]
print "There are "+ str(len(duplicates)) + " duplicate records found"
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

In [ ]:
df = df.drop_duplicates(cols, keep='last')
df.shape

In [ ]:
# check duplication
cols = ['bike','time_start'] # this should uniquely identified a record, as the same bike could not be used by multiple trip started at the same time 
duplicates = df[df.duplicated(cols,keep=False)]
print "There are "+ str(len(duplicates)) + " duplicate records found"
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

From the recrods, seems everything is equvalent, but how come it was not found from the beginning?
Hints: Comparing floats are always tricky!

In [ ]:
# convert duaration to int
df.duration = df.duration.astype(int)
duplicates = df[df.duplicated(df.columns,keep=False)] # try to identify duplicates based on all records
print "There are "+ str(len(duplicates)) + " duplicate records found"
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

In [ ]:
df = df.drop_duplicates(cols, keep='last')
df.shape

## V. Save integrated data

In [ ]:
df.to_csv("bikeshare_data_integrate.csv",index = False)

### Note:
- Ideally, once all schema conflicts are perfectly resolved, we can simply identify duplicate using the complete records. However, realistically, this will not happen. 
- Integration might often become an interative process, i.e., we might find out more conflicts to be resolve at a later stage of integration. 
- Carefully examine and identify global key is a good practice for identifying data conflicts in integration. Although, domain knowledge and analysis are required.